In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.metrics import accuracy_score, mean_squared_error

import tensorflow
from tensorflow import keras


In [ ]:
df = pd.read_csv('/content/Crop_recommendation.csv')
df=df.drop('N',axis=1)
df=df.drop('P',axis=1)
df=df.drop('K',axis=1)
df=df.drop('ph',axis=1)

In [ ]:
df

,temperature,humidity,rainfall,label
0,20.879744,82.002744,202.935536,rice
1,21.770462,80.319644,226.655537,rice
2,23.004459,82.320763,263.964248,rice
3,26.491096,80.158363,242.864034,rice
4,20.130175,81.604873,262.717340,rice
...,...,...,...,...
2195,26.774637,66.413269,177.774507,coffee
2196,27.417112,56.636362,127.924610,coffee
2197,24.131797,67.225123,173.322839,coffee
2198,26.272418,52.127394,127.175293,coffee


In [ ]:
X = df.drop(['label'],axis=1)
y = df['label']
print(len(set(y)))

22


In [ ]:
le = LabelEncoder()
y = le.fit_transform(y)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
model = Sequential()
model.add(Dense(units=32, activation='relu', input_dim=len(X_train.columns)))
model.add(Dense(units=64, activation='relu'))

In [ ]:
num_classes = len(set(y))
model.add(Dense(units=num_classes, activation='softmax'))


In [ ]:
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
model.fit(X_train, y_train, epochs=100, batch_size=32)

Epoch 1/100
55/55 [==============================] - 1s 2ms/step - loss: 14.1087 - accuracy: 0.0699
Epoch 2/100
55/55 [==============================] - 0s 2ms/step - loss: 2.3420 - accuracy: 0.2670
Epoch 3/100
55/55 [==============================] - 0s 2ms/step - loss: 1.9648 - accuracy: 0.3330
Epoch 4/100
55/55 [==============================] - 0s 2ms/step - loss: 1.8277 - accuracy: 0.3614
Epoch 5/100
55/55 [==============================] - 0s 2ms/step - loss: 1.6670 - accuracy: 0.3875
Epoch 6/100
55/55 [==============================] - 0s 2ms/step - loss: 1.5886 - accuracy: 0.4068
Epoch 7/100
55/55 [==============================] - 0s 2ms/step - loss: 1.5104 - accuracy: 0.4227
Epoch 8/100
55/55 [==============================] - 0s 2ms/step - loss: 1.4757 - accuracy: 0.4528
Epoch 9/100
55/55 [==============================] - 0s 2ms/step - loss: 1.4104 - accuracy: 0.4881
Epoch 10/100
55/55 [==============================] - 0s 2ms/step - loss: 1.4474 - accuracy: 0.4739
Epoch 11

In [ ]:
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f'Test Accuracy: {test_acc}')


14/14 [==============================] - 0s 2ms/step - loss: 0.9027 - accuracy: 0.6386
Test Accuracy: 0.6386363506317139


In [ ]:

y_hat = model.predict(X_test)
y_hat_classes = y_hat.argmax(axis=1)

14/14 [==============================] - 0s 1ms/step


In [ ]:

accuracy = accuracy_score(y_test, y_hat_classes)
print(f'Accuracy: {accuracy}')



Accuracy: 0.6386363636363637


In [ ]:
new_df=pd.DataFrame([[13.02489,50.98861,88.55123]],columns=X.columns)

predictions = model.predict(new_df)

predicted_classes = predictions.argmax(axis=1)
print(predicted_classes)
predicted_labels = le.inverse_transform(predicted_classes)


1/1 [==============================] - 0s 21ms/step
[8]


In [ ]:
print(predicted_labels)

['jute']


In [ ]:
keras_file="linear.h5"
keras.models.save_model(model,keras_file)

<ipython-input-35-39bb9e0d703a>:2: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  keras.models.save_model(model,keras_file)


In [ ]:

converter=tensorflow.lite.TFLiteConverter.from_keras_model(model=model)
model_tflite=converter.convert()
open("linearRegrssionModel.tflite","wb").write(model_tflite)

16804

# New Section